<a href="https://colab.research.google.com/github/Galium-aparine/CS49200_ML_Recipe_Generator/blob/maddie/RecipeGenerator_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile RecipeGenerator.py
# Installing requirements

# Pre-trained recipe generator model
!pip install transformers
# Framework for app
!pip install streamlit

import streamlit as st
import random
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer

# ----------------------
# Streamlit App Setup
# ----------------------

st.set_page_config(page_title="Smart Recipe Generator", page_icon="🍽️")
st.title("Recipe Generator")
st.markdown("Upload a picture of your ingredients (coming soon) or type them to get personalized recipes!")

# ----------------------
# User Input Section
# ----------------------

# Ingredient input
input_method = st.radio("Choose input method:", ["Type ingredients", "Upload a photo (coming soon)"])

ingredients = []
if input_method == "Type ingredients":
    ingredients_text = st.text_area("Enter ingredients (comma-separated):", placeholder="e.g., chicken, tomatoes, rice")
    if ingredients_text:
        ingredients = [i.strip().lower() for i in ingredients_text.split(",")]

# Preferences and filters
st.header("Preferences & Filters")
allergies = st.text_input("Any allergies? (comma-separated, optional):").lower().split(",")
cuisine_filter = st.selectbox("Preferred Cuisine:", options=["Any", "Italian", "Indian", "American", "Mexican", "Chinese"])
max_time = st.slider("Maximum Cooking Time (minutes):", min_value=10, max_value=120, value=60)

# ----------------------
# Recipe Generator (for now)
# ----------------------

# def recipe_generator(ingredients, allergies, cuisine, max_time):
MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

prefix = "items: "
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}

def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe

# ----------------------
# Generate and Display Recipes
# ----------------------

if st.button("Find Recipes"):
    if ingredients:
        st.header("Recommended Recipes 🍳")
        generated = generation_function(ingredients)

        for text in generated:
            sections = text.split("\n")
            headline = ""
            for section in sections:
                section = section.strip()
                if section.startswith("title:"):
                    section = section.replace("title:", "")
                    headline = "TITLE"
                elif section.startswith("ingredients:"):
                    section = section.replace("ingredients:", "")
                    headline = "INGREDIENTS"
                elif section.startswith("directions:"):
                    section = section.replace("directions:", "")
                    headline = "DIRECTIONS"

                if headline == "TITLE":
                    st.subheader(section.strip().capitalize())
                else:
                    section_info = [f"  - {i+1}: {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
                    st.subheader(headline)
                    st.write("\n".join(section_info))

                # st.image(recipe['image'], width=300)

        st.write("-" * 130)
    else:
        st.warning("Please enter ingredients to generate recipes!")



Writing RecipeGenerator.py
